In [ ]:
#Dense fake_data
#Simulating the Jij XX oscillations with active heating + maybe dephasing noise
"""How to deal with active heating of the chain?
Option 1: increase the random theta linearly in time. Probably the best bet. 
Option 2: rescample ever few time steps with a higher temperature.

I think option 1 is the best for now
"""
sx, sy, sz = get_s_lists(2)

omega_sim = Omega[0]
rabi_freq = np.sqrt(omega_sim)

decay_envelope = lambda t: np.exp(-t*0.25)
time_decay_factors = decay_envelope(times)
#dephasing:
#rate = 0
#rate = 0
noisy_ops = [sx[0], sx[1]]
obs = [0.5*(sz[0] - sz[1])]

HXX = sx[0] * sx[1]
#HXX = HXX* Omega**2

def genH(theta_0, theta_dot, Omega):
    HXX = sx[0] * sx[1]

    HXX = HXX * Omega/2
    delta = thermal_rejection(theta_0,1)[0]
    delta=0
    return HXX*((1-2*delta))


psi0 = tensor(basis(2,0), basis(2,1))

#import Options from qutip:
from qutip.solver import Options
#increase nsteps
options = Options(nsteps=1e3)

def run(theta_0, theta_dot,Omega):
    #print('running')
    H = genH(theta_0,theta_dot,Omega)
    results = mesolve(H, psi0,times, [i*theta_0 for i in noisy_ops], obs, options=options).expect[0]#, ntraj=1)
    #print(results)
    #print(results)
    return np.array(results)
shots = 1


#full_results = Parallel(n_jobs=6)(delayed(run)(theta_0, theta_dot,Omega) for i in range(shots))
    

def gen_full_results(theta_0, theta_dot,Omega):
    #H = genH(theta_0, theta_dot)
    full_results = np.array(Parallel(n_jobs=6)(delayed(run)(theta_0, theta_dot,Omega) for _ in range(shots)))
    #print(full_results.shape)
    averaged = np.zeros(len(times))
    #print(averaged.shape)
    for item in full_results:
        #print(item.expect[0])
        averaged += item
    averaged = averaged/shots
    
    return averaged#[indices]


In [ ]:
#For each theta in dense_theta, pair it with dense_rescale, and generate the output from gen_ful_results. Combine into a matrix called dense_sim_flopping. 
dense_theta = np.linspace(0, 0.5, 10)
dense_sim_flopping = np.zeros((len(times), len(dense_theta)))
print(dense_sim_flopping[:,0].shape)
for i in range(len(dense_theta)):
    result = gen_full_results(dense_theta[i], theta_dot, Omega[0])#*correction*(dense_rescale[i]**2*dense_rescale[i]/dense_rescale[i]))
    print(result.shape)
    dense_sim_flopping[:,i] = result
#multiply the time axis by the time_decay_factors
print(dense_theta.shape)


In [ ]:
#blue to red:
cmap = LinearSegmentedColormap.from_list("mycmap", ["blue", "red"])
#Set the font size to 24:
plt.rcParams.update({'font.size': 24})
fig, (ax2,ax) = plt.subplots(2,1, figsize=(20,15))

experimental_limit = 5

for i in range(experimental_limit,10):
    ax.plot(times, dense_sim_flopping[:,i], color = cmap(i/10))
ax.plot(times, dense_sim_flopping[:,0], color = 'gray', label='Noise Free Limit', linestyle='--', lw=5)
#Add a color bar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=max(dense_theta)))
#sm.set_label(r'Temperature ($\lambda$)')
#Add the color bar to the plot:
ax.set_xlabel('Time (ms)')
ax.set_ylabel(r'Expectation Value: $\langle Z_0\rangle - \langle Z_1\rangle$')
cbar = plt.colorbar(sm, ax=ax)

cbar.set_label(r'Noise Rate ($\lambda$)')
ax2.scatter(dense_theta[:experimental_limit], dense_sim_flopping[25,:experimental_limit], color = 'gray', label='Sub Experimental Limit')
ax2.scatter(dense_theta[experimental_limit:], dense_sim_flopping[25,experimental_limit:], color = 'black', marker='o', s=100, label='Error Scaled Data')
ax2.vlines(dense_theta[experimental_limit], 0.4, 1, linestyle='--', color = 'black')
ax2.text(0, 0.45, 'Best Experimental Value', rotation=0, size=30)

#Fit to third_no_first:
fit_func = third_no_first_ZNE(dense_theta[experimental_limit:], dense_sim_flopping[25,experimental_limit:])
ax2.plot(dense_theta, fit_func(dense_theta), color = 'blue', label = "Fit of Experimental Data")
ax2.legend()
ax2.set_xlabel(r'Noise Rate ($\lambda$)')
ax2.set_ylabel(r'Expectation Value: $\langle Z_0\rangle - \langle Z_1\rangle$')
ax.set_title('Dynamics vs. Noise Rate')
ax2.set_title('Extrapolation across Noise Rates')

#Compute ZNE for each time:
ZNE_data = np.zeros(30)
for i in range(30):
    fit_func = third_no_first_ZNE(dense_theta[experimental_limit:], dense_sim_flopping[i,experimental_limit:])
    ZNE_data[i] = fit_func(0)

ax.plot(times, ZNE_data, label = 'ZNE Limit', color = 'blue', zorder=-1)
ax.legend()

fig.tight_layout()

ax.vlines(times[25], -1,1, linestyle='--', color = 'black')

plt.savefig('lindblad_ZNE_Demo.png', dpi=300)

In [ ]:
#blue to red:
cmap = LinearSegmentedColormap.from_list("mycmap", ["blue", "red"])
#Set the font size to 24:
plt.rcParams.update({'font.size': 16})
fig, (ax2) = plt.subplots(1, figsize=(10,6))

experimental_limit = 5


#Add a color bar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=max(dense_theta)))
#sm.set_label(r'Temperature ($\lambda$)')
#Add the color bar to the plot:


ax2.scatter(dense_theta[:experimental_limit], dense_sim_flopping[25,:experimental_limit], color = 'red', label='Sub Experimental Limit')
ax2.scatter(dense_theta[experimental_limit:], dense_sim_flopping[25,experimental_limit:], color = 'black', marker='o', s=100, label='Error Scaled Data')
ax2.vlines(dense_theta[experimental_limit], 0.4, 1, linestyle='--', color = 'black')
ax2.text(0.06, 0.45, 'Best Experimental Value', rotation=0, size=20)

#Fit to third_no_first:
fit_func = third_no_first_ZNE(dense_theta[experimental_limit:], dense_sim_flopping[25,experimental_limit:])
ax2.plot(dense_theta, fit_func(dense_theta), color = 'blue', label = "Fit of Experimental Data")
ax2.legend()
ax2.set_xlabel(r'Noise Rate ($\lambda$)')
ax2.set_ylabel(r'Expectation Value: $\langle Z_0\rangle - \langle Z_1\rangle$')
ax2.set_title('Zero Noise Extrapolation: XX(t) Gate')

#Compute ZNE for each time:
ZNE_data = np.zeros(30)
for i in range(30):
    fit_func = third_no_first_ZNE(dense_theta[experimental_limit:], dense_sim_flopping[i,experimental_limit:])
    ZNE_data[i] = fit_func(0)


fig.tight_layout()


plt.savefig('lindblad_ZNE_Demo_lambda.png', dpi=300)